# Text Analytics with 3-1-1 Call Data
> An attempt at text analytics using non-emergency service calls.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/cincy_skyline.jpg

## Flow
- overview of TF-IDF
- remove stop words
- look at index of % services by department cut by neighborhood/time to showcase how we learn more from text analytics
- look at top words overall (these might be automated responses, if so remove)
- run tf-idf by neighborhood for set time period
     - explore differences between Hyde Park and Avondale
- run tf-idf by month for all neighbordhoods over set time period
     - explore differences between a single month per season


# About

Text analytics represents a growing set of tools to uncover insights on texts of interest.  In industry, this may include mining customer reviews or .

The set of tools is strong and enables us to 'listen' to people in a programmatic way.  Text analytics obviously has limitations compared to the understanding that comes from an actual human reading the text, but for processing large amounts of data it can be an effective tool within the confines of its limitations.

## In this Post

This notebook will demonstrate various text mining techniques, with our subject being 3-1-1 call data.  3-1-1 Call Centers are common in municipalities and represent a non-emergency hotline for residents to contact.  Incidents reported include fallen limbs, potholes, overgrown footprints, and much more. 

Text mining was a new topic for me.  I have been exposed to trainings in the past but have never wandered into the wild lands of raw text descriptions and tried to make better sense of them programmatically.  This ended up being a fun exercise with a ton of text cleaning and I learned a lot - I hope you do as well!

## Why is this Important

Querying and accessing data is a fundamental step of any data science workflow.  The mechanics of accessing data can get very messy, especially when it comes to government data.  Examples of this messiness include manually downloading files and saving to a user-defined location, appending multiple Excel files, scraping PDFs, and the list goes on.  Bottom line: it can get MESSY.

Luckily, the infrastructure provided by Socrata makes for a seamless experience that can be replicated by anyone with the Internet and Python.  You read that correctly, everything demonstrated in this post can be replicated with very little setup.  This is a huge perk for the sake of collaboration.  Additionally, in the event that an analysis needs to be re-run, perhaps on more recent data, the ability to query data programatically allows for minimal rework and minimal room for error.  Alright, enough hype let's get to it!

## Setup

Before we get rolling, it is important to be working in an environment with the necessary packages installed and available.  For this post we will need the `pandas`, `datetime` and `nltk` libraries installed.  As mentioned before, I like using conda to manage dependencies and would encourage others to go that route.

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import plotly.express as px

pd.options.display.max_colwidth = 100

# function takes five arguments (endpoint_url and 4 components of the query) 
# function returns a full, cleaned up API call
def generate_query(endpoint_url, query, limit):

    raw_query = (f"{endpoint_url}?$query="
                 f"{query}%20"
                 f"limit {limit}"
                )
    
    # get rid of control characters
    for replacements in ((" ", "%20"), ("\n", "%20")):
        raw_query = raw_query.replace(*replacements)
    
    return raw_query


## Initial Read

We are going to read in the past year of 3-1-1 call data for this exploration.

In [2]:
# define our base API endpoint
endpoint_311 = 'https://data.cincinnati-oh.gov/resource/4cjh-bm8b.json'

# dynamically generate today's date
today = dt.date.today()

# dynamically arrive at the date exactly 365 days ago
year_ago_today = today - dt.timedelta(days = 365)

Use our function to build a query.

In [3]:
pd.read_json(endpoint_311).head(5)

,jurisdiction_id,service_request_id,status,service_name,service_code,description,agency_responsible,requested_datetime,updated_datetime,expected_datetime,address,zipcode,latitude,longitude,requested_date,updated_date,last_table_update
0,CINCINNATI,SR14009513,CLOS,"""Metal Furniture, Spec Collectn""","""MTL-FRN""","""THere will also be a vacuum cleaner, and carpet cleaner for pick up""",Public Services,2014-02-05T23:21:00Z,2014-02-28T00:00:00Z,2014-02-19T00:00:00Z,"""4601 CRAWFORD AV, CINC - GJ1298238864""",45223,39.172947,-84.534519,2014-02-05T00:00:00.000,2014-02-28T00:00:00.000,2015-03-05T23:07:49.000
1,CINCINNATI,SR14009514,CLOS,"""Sign, street sign faded""","""STSGN""","""southwest corner of intersection""",Public Services,2014-02-05T23:21:00Z,2014-02-13T00:00:00Z,2014-02-06T00:00:00Z,"""MAIN ST & WOODWARD ST""",45202,39.110518,-84.511968,2014-02-05T00:00:00.000,2014-02-13T00:00:00.000,2015-03-05T23:07:49.000
2,CINCINNATI,SR14009515,CLOS,"""Slippery streets, request""","""SLPYST""","""Request entered through the Web. Refer to Intake Questions for further description.""",Public Services,2014-02-05T23:23:00Z,2014-03-07T00:00:00Z,2014-02-06T00:00:00Z,"""2779 MORNINGRIDGE DR, CINC - GJ0839534632""",45211,39.135951,-84.588884,2014-02-05T00:00:00.000,2014-03-07T00:00:00.000,2015-03-05T23:07:49.000
3,CINCINNATI,SR14009516,CLOS,"""Property damage, traffic aids""","""PRDMTAID""","""Transfer: 02/06/2014 6:43 AM/DCOTTRELLPlease check, mailbox was distroyed by snow plow - Reques...",Public Services,2014-02-05T23:24:00Z,2014-03-27T00:00:00Z,2014-02-20T00:00:00Z,"""4601 CRAWFORD AV, CINC - GJ1298238864""",45223,39.172947,-84.534519,2014-02-05T00:00:00.000,2014-03-27T00:00:00.000,2015-03-05T23:07:49.000
4,CINCINNATI,SR14009517,CLOS,"""Graffiti, removal""","""GRFITI""","""Request entered through the Web. Refer to Intake Questions for further description.""",Public Services,2014-02-05T23:28:00Z,2014-02-13T00:00:00Z,2014-03-07T00:00:00Z,"""1227 MAIN ST, CINC - GJ1512631874""",45202,39.109491,-84.511949,2014-02-05T00:00:00.000,2014-02-13T00:00:00.000,2015-03-05T23:07:49.000


Aggregate total counts of requests by `agency_responsible` for the past year.

In [4]:
total_agency_responsible_query = (
    
    generate_query(endpoint_url = endpoint_311, 
                   query = f"""select agency_responsible, 
                                      count(*) as n
                               where requested_datetime>='{year_ago_today}' 
                               and requested_datetime<='{today}'
                               group by agency_responsible""", 
                   limit = 1000)
)

Grouping by `agency_responsible` we see that the bulk of our records lie in 'Public Services' and a few other departments.  In lieu of this information, let's only investigate the Public Services records and break it down by these mysterious `service_code` values.

In [5]:
agency_responsible_count = pd.read_json(total_agency_responsible_query)

agency_responsible_count \
    .assign(total_requests=lambda x: np.sum(x['n']),
            pct_total=lambda x: x['n'] / x['total_requests']) \
    .sort_values(by='pct_total', ascending = False).head(10)

,agency_responsible,n,total_requests,pct_total
13,Public Services,74969,103266,0.725980
4,Cinc Building Dept,6047,103266,0.058558
1,City Manager's Office,5695,103266,0.055149
6,Cinc Health Dept,5057,103266,0.048971
2,Police Department,4932,103266,0.047760
15,Dept of Trans and Eng,3866,103266,0.037437
16,Park Department,786,103266,0.007611
8,Cin Water Works,540,103266,0.005229
9,Fire Dept,468,103266,0.004532
3,Metropolitan Sewer,381,103266,0.003690


#### Filter to Public Services only and aggregate the counts across the entire city

In [6]:
total_services_query = (
    
generate_query(endpoint_url = endpoint_311, 
               query = f"""select service_code, 
                                  service_name, 
                                  count(*) as n 
                           where requested_datetime>='{year_ago_today}' 
                           and requested_datetime<='{today}' 
                           and agency_responsible == 'Public Services'
                           group by service_code, 
                                    service_name""", 
               limit = 1000)

)

In [7]:
total_services = pd.read_json(total_services_query) \
    .assign(total_n=lambda x: np.sum(x['n']),
            pct_total=lambda x: x['n'] / x['total_n']) \
    .sort_values(by = 'pct_total', ascending = False)

total_services.head(10)

,service_code,service_name,n,total_n,pct_total
46,"""MTL-FRN""","""Metal Furniture, Spec Collectn""",27088,74969,0.361323
77,"""RF-COLLT""","""Trash, request for collection""",5757,74969,0.076792
85,"""YDWSTA-J""","""Yard waste,rtc""",3744,74969,0.049941
70,"""LITR-PRV""","""Litter, private property""",3643,74969,0.048593
36,"""PTHOLE""","""Pothole, repair""",3173,74969,0.042324
99,"""TLGR-PRV""","""Tall grass/weeds, private prop""",3061,74969,0.040830
79,"""SLPYST""","""Slippery streets, request""",2712,74969,0.036175
63,"""TGGDCLLC""","""Trash, tagged collections""",2495,74969,0.033280
45,"""TRSHCRTR""","""Trash cart, registration""",2349,74969,0.031333
41,"""TIRES""","""Tires, Special Collection""",1640,74969,0.021876


In [8]:
zipcode_services_query = (
    
generate_query(endpoint_url = endpoint_311, 
               query = f"""select zipcode,
                                  service_code, 
                                  service_name, 
                                  count(*) as n
                           where requested_datetime>='{year_ago_today}' 
                           and requested_datetime<='{today}' 
                           and agency_responsible == 'Public Services'
                           group by zipcode,
                                    service_code, 
                                    service_name""", 
               limit = 1000)

)

In [9]:
zipcode_services = pd.read_json(zipcode_services_query) \
    .groupby('zipcode') \
    .apply(lambda x: x.assign(total_n=np.sum(x['n']))) \
    .assign(pct_total=lambda x: x['n'] / x['total_n'])


zipcode_services \
    .sort_values(by='pct_total', ascending = False).head(10)

,,zipcode,service_code,service_name,n,total_n,pct_total
zipcode,,,,,,,
45251.0,676,45251.0,"""DMGNOD""","""Damage Claim - NOD""",1,1,1.000000
45244.0,731,45244.0,"""PTHOLE""","""Pothole, repair""",1,1,1.000000
45241.0,389,45241.0,"""TRSHREMV""","""Trash cart, remove""",1,1,1.000000
45230.0,718,45230.0,"""MTL-FRN""","""Metal Furniture, Spec Collectn""",1273,1859,0.684777
45220.0,57,45220.0,"""MTL-FRN""","""Metal Furniture, Spec Collectn""",847,1255,0.674900
45236.0,248,45236.0,"""RF-COLLT""","""Trash, request for collection""",4,6,0.666667
45215.0,912,45215.0,"""LITR-PRV""","""Litter, private property""",43,71,0.605634
45219.0,573,45219.0,"""MTL-FRN""","""Metal Furniture, Spec Collectn""",1123,1855,0.605391
45221.0,604,45221.0,"""PTHOLE""","""Pothole, repair""",3,5,0.600000


In [10]:
zipcode_services.merge(total_services, on=['service_code','service_name'], how='left') \
    .assign(service_index=lambda x: x['pct_total_x'] / x['pct_total_y']) \
    .query("n_x > 20") \
    .sort_values(by='service_index', ascending=False) \
    .query("zipcode == 45202") \
    .head(20)

,zipcode,service_code,service_name,n_x,total_n_x,pct_total_x,n_y,total_n_y,pct_total_y,service_index
12,45202.0,"""COVID_19""","""General Inquiry""",29,1504,0.019282,31,74969,0.000414,46.630512
14,45202.0,"""CRNRCNOF""","""Corner can, overflowing""",48,1504,0.031915,357,74969,0.004762,6.702038
26,45202.0,"""TLGR-PS""","""Tall grass/weeds, PS property""",42,1504,0.027926,495,74969,0.006603,4.229392
1,45202.0,"""DUMP-PVS""","""Dumping, prv prop <2500 sq ft""",82,1504,0.054521,1073,74969,0.014313,3.809325
10,45202.0,"""RWFRNTRT""","""ROW furniture/trash dumping""",47,1504,0.031250,692,74969,0.009230,3.385522
38,45202.0,"""LITR-PRV""","""Litter, private property""",226,1504,0.150266,3643,74969,0.048593,3.092311
11,45202.0,"""PTHOLE""","""Pothole, repair""",188,1504,0.125000,3173,74969,0.042324,2.953396
22,45202.0,"""YDWSTA-J""","""Yard waste,rtc""",193,1504,0.128324,3744,74969,0.049941,2.569540
16,45202.0,"""SGNDNA""","""Sign, down/missing stop sign""",22,1504,0.014628,467,74969,0.006229,2.348225
21,45202.0,"""RF-COLLT""","""Trash, request for collection""",265,1504,0.176197,5757,74969,0.076792,2.294476


In [132]:
raw_descriptions_query = (
    
generate_query(endpoint_url = endpoint_311, 
               query = f"""select zipcode,
                                  requested_datetime,
                                  service_code,
                                  service_name,
                                  description
                            where requested_datetime>='{year_ago_today}' 
                            and requested_datetime<='{today}' 
                            and agency_responsible == 'Public Services'""",
               limit = 100000000000)

)

In [133]:
# get rid of 
# get rid of quotation marks in the string
# filter out blank description records
raw_311 = pd.read_json(raw_descriptions_query) \
    .assign(description_clean = lambda x: x['description'].str.replace('[^a-zA-Z\s]', '')
                                                          .str.replace('"', '')
                                                          .str.replace('  ', ' ')
                                                          .str.lower(),
            service_code = lambda x: x['service_code'].str.replace('"', ''),
            service_name = lambda x: x['service_name'].str.replace('"', '')) \
    .query("description_clean != 'request entered through the web refer to intake questions for further description'") \
    .query("description_clean != ' '")

/Users/joebahr/opt/miniconda3/envs/data-science-with-r/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [134]:
raw_311

,zipcode,requested_datetime,service_code,service_name,description,description_clean
9,45205.0,2020-10-08T07:17:00Z,DAPUB1,Dead animal,"""DEAD ANIMAL RACOON ON W 8TH NEAR SUNSET IN CURB LANE""",dead animal racoon on w th near sunset in curb lane
11,45224.0,2020-10-08T07:25:00Z,TGGDCLLC,"Trash, tagged collections","""TRASH NOT OUT @ 0709am""",trash not out am
13,45224.0,2020-10-08T07:26:00Z,TGGDCLLC,"Trash, tagged collections","""TAGGED @ 0709am TOOK TOTER & (3) BAGS/ITEMS - LEFT THE REST""",tagged am took toter bagsitems left the rest
14,45223.0,2020-10-08T07:28:00Z,RF-COLLT,"Trash, request for collection","""TRASH NOT OUT @ 0715am\n10/08/2020 3:09 PM/RMCCRAY - LSO- CART AT THE CURB\n10/12/2020 4:52 PM/...",trash not out am\n pmrmccray lso cart at the curb\n pmsgoodwin trash still not collected
15,45239.0,2020-10-08T07:28:00Z,DAPUB1,Dead animal,"""DEAD RACOON IN WHITE TRASH BAG AT THE CURB""",dead racoon in white trash bag at the curb
...,...,...,...,...,...,...
74820,45224.0,2021-09-17T15:47:00Z,TLGR-PRV,"Tall grass/weeds, private prop","""3 FT TALL GRASS""",ft tall grass
74823,45238.0,2021-09-17T16:05:00Z,SVCCMPLT,"Service complaint, trash","""CITIZEN REQUESTING A CALL BACK REGARDING TRASH COLLECTION. CITIZEN CALL IN MONDAY AFTER TRASH W...",citizen requesting a call back regarding trash collection citizen call in monday after trash was...
74824,45202.0,2021-09-17T16:11:00Z,RF-COLLT,"Trash, request for collection","""rtc""",rtc
74825,45230.0,2021-09-17T16:12:00Z,PTHOLE,"Pothole, repair","""POT HOLE REPAIR - ON BEECHMONT EASTBOUND 10 YARDS EAST OF CORBLY IN CURB LANE """,pot hole repair on beechmont eastbound yards east of corbly in curb lane


In [135]:
# concatenate all strings within the same zipcode
zipcode_descriptions = raw_311.groupby(['zipcode'])['description_clean'].apply(' '.join).reset_index()

In [145]:
?TfidfVectorizer

In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(input = 'content', stop_words='english', smooth_idf = True)

tfidf_vector = tfidf_vectorizer.fit_transform(zipcode_descriptions['description_clean'])


In [158]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), 
                        index=zipcode_descriptions['zipcode'], 
                        columns=tfidf_vectorizer.get_feature_names()).reset_index()

tfidf_df

,zipcode,aa,aaron,aas,ab,abandoned,abanoned,abatement,abgle,abigail,...,yw,zier,zinsle,zip,zips,zone,zoning,zoo,zoom,zula
0,45002.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
1,45202.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008696,...,0.029060,0.000000,0.000000,0.000000,0.00000,0.005697,0.000000,0.00000,0.008696,0.000000
2,45203.0,0.000000,0.027097,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.019183,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
3,45204.0,0.000000,0.000000,0.00000,0.000000,0.002889,0.000000,0.000000,0.000000,0.000000,...,0.026668,0.000000,0.000000,0.000000,0.00000,0.003921,0.000000,0.00000,0.000000,0.000000
4,45205.0,0.000000,0.000000,0.00000,0.002162,0.002322,0.000000,0.000000,0.002162,0.000000,...,0.035966,0.000000,0.000000,0.000000,0.00000,0.000000,0.007213,0.00000,0.000000,0.000000
5,45206.0,0.000000,0.000000,0.00000,0.000000,0.002292,0.000000,0.000000,0.000000,0.000000,...,0.045321,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
6,45207.0,0.000000,0.000000,0.00000,0.000000,0.003162,0.000000,0.000000,0.000000,0.000000,...,0.029188,0.000000,0.000000,0.000000,0.00000,0.004292,0.000000,0.00000,0.000000,0.000000
7,45208.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.084167,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
8,45209.0,0.007376,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.041774,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
9,45211.0,0.000000,0.000000,0.00000,0.000000,0.002752,0.000000,0.000000,0.000000,0.000000,...,0.045355,0.000000,0.000000,0.000000,0.00000,0.003735,0.000000,0.00000,0.000000,0.000000


In [160]:
tfidf_df \
    .melt(id_vars = 'zipcode', value_vars = tfidf_df.columns).sort_values(by = 'value', ascending = False) \
    .query("variable == 'trash'")

,zipcode,variable,value
301905,45229.0,trash,0.489658
301886,45207.0,trash,0.477894
301884,45205.0,trash,0.462905
301913,45239.0,trash,0.448259
301889,45211.0,trash,0.433979
301911,45237.0,trash,0.426800
301912,45238.0,trash,0.410916
301892,45214.0,trash,0.408379
301883,45204.0,trash,0.398589
301900,45224.0,trash,0.396493
